In [37]:
#import pandas as pd
!wget http://www-public.tem-tsp.eu/~zhang_da/pub/dataset_tsmc2014.zip
!unzip dataset_tsmc2014.zip

--2024-05-28 13:13:41--  http://www-public.tem-tsp.eu/~zhang_da/pub/dataset_tsmc2014.zip
Resolving www-public.tem-tsp.eu (www-public.tem-tsp.eu)... 157.159.10.107, 2001:660:3203:100:1:0:80:107
Connecting to www-public.tem-tsp.eu (www-public.tem-tsp.eu)|157.159.10.107|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25546284 (24M) [application/zip]
Saving to: ‘dataset_tsmc2014.zip.3’

dataset_tsmc2014.zi 100%[===================>]  24.36M  5.16MB/s    in 7.4s    

2024-05-28 13:13:49 (3.29 MB/s) - ‘dataset_tsmc2014.zip.3’ saved [25546284/25546284]

Archive:  dataset_tsmc2014.zip
replace dataset_tsmc2014/dataset_TSMC2014_NYC.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [38]:
import pandas as pd
columns = ['User ID',
 'Venue ID',
 'Venue category ID',
 'Venue category name',
 'Latitude',
 'Longitude',
 'Timezone',
 'UTC time']

In [39]:
NUMCHECKIN = 300

df = pd.read_csv('dataset_tsmc2014/dataset_TSMC2014_NYC.txt', sep='\t', encoding='latin-1', names=columns)
df = df.sample(NUMCHECKIN)
print(df)

        User ID                  Venue ID         Venue category ID  \
152430      860  4aeb4a99f964a52096c021e3  4bf58dd8d48988d16e941735   
117404      651  4daa6264a86e771ea723a76c  4bf58dd8d48988d1fc941735   
193116      201  4ace6c89f964a52078d020e3  4bf58dd8d48988d1ed931735   
202223      736  43a52546f964a520532c1fe3  4bf58dd8d48988d1ed931735   
207041      154  4a6d9e12f964a520cad21fe3  4bf58dd8d48988d114951735   
...         ...                       ...                       ...   
152603      981  4f72095ae4b007de90522a69  4bf58dd8d48988d1e0931735   
151901      876  42911d00f964a520f5231fe3  4bf58dd8d48988d129951735   
69050       357  4af1ae9cf964a52026e221e3  4bf58dd8d48988d124941735   
65469       335  4b868537f964a520bc8d31e3  4bf58dd8d48988d1f9931735   
5658        238  4db44994cda1c57c82583709  4bf58dd8d48988d1f1931735   

          Venue category name   Latitude  Longitude  Timezone  \
152430   Fast Food Restaurant  40.746050 -73.990593      -240   
117404        Lau

In [40]:
from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()
df['Venue ID'] = le.fit_transform(df['Venue ID'])
df['Venue category ID'] = le.fit_transform(df['Venue category ID'])
df.drop(['Venue category name'],axis=1,inplace=True)
print(df)

        User ID  Venue ID  Venue category ID   Latitude  Longitude  Timezone  \
152430      860        88                 59  40.746050 -73.990593      -240   
117404      651       214                107  40.832115 -73.909565      -240   
193116      201        84                 97  40.773839 -73.871220      -300   
202223      736        19                 97  40.645089 -73.784523      -300   
207041      154        62                 20  40.631796 -73.947678      -300   
...         ...       ...                ...        ...        ...       ...   
152603      981       266                 90  40.764628 -73.988142      -240   
151901      876        17                 35  40.750795 -73.993576      -240   
69050       357        92                 30  40.743220 -74.007734      -240   
65469       335       122                103  40.584757 -73.950348      -240   
5658        238       215                 98  40.739398 -73.993210      -240   

                              UTC time 

In [41]:
import math

def distance_between_coordinates(lat1, lon1, lat2, lon2):
    radius = 6371  # Radius of the Earth in km

    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Calculate the differences in latitude and longitude<
    d_lat = lat2 - lat1
    d_lon = lon2 - lon1

    # Calculate the distance using the Haversine formula
    a = math.sin(d_lat/2) * math.sin(d_lat/2) + math.cos(lat1) * math.cos(lat2) * math.sin(d_lon/2) * math.sin(d_lon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = radius * c

    return distance

In [42]:
print(distance_between_coordinates(40.719810,-74.002581,40.735981, -74.029309))

2.8819941850086326


In [43]:
#build an alternative temporal graph
#two pois are connected if users go on average to them in the seme timeframe

df['localTime'] = pd.to_datetime(df['UTC time']) + pd.to_timedelta(df['Timezone'], unit = 'm')

df['localDayofWeek'] = df.localTime.dt.dayofweek
df['localHour'] = df.localTime.dt.hour






<ipython-input-43-f85843e15d2c>:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['localTime'] = pd.to_datetime(df['UTC time']) + pd.to_timedelta(df['Timezone'], unit = 'm')


In [44]:
venuesdf = df[['Venue ID','Latitude','Longitude','Venue category ID']].drop_duplicates(subset=['Venue ID'])
numvenues = len(venuesdf)
print(venuesdf)


        Venue ID   Latitude  Longitude  Venue category ID
152430        88  40.746050 -73.990593                 59
117404       214  40.832115 -73.909565                107
193116        84  40.773839 -73.871220                 97
202223        19  40.645089 -73.784523                 97
207041        62  40.631796 -73.947678                 20
...          ...        ...        ...                ...
47788         82  40.741570 -73.987385                 30
109795       208  40.632175 -73.918346                  3
152603       266  40.764628 -73.988142                 90
69050         92  40.743220 -74.007734                 30
65469        122  40.584757 -73.950348                103

[286 rows x 4 columns]


In [45]:
#!pip3 install torch --index-url https://download.pytorch.org/whl/${CUDA}
import torch
print(torch.__version__)
print(torch.version.cuda)
!pip install torch-geometric \
  torch-sparse \
  torch-scatter \
  torch-cluster \
  -f https://pytorch-geometric.com/whl/torch-2.3.0+cu121.html

2.3.0+cu121
12.1
Looking in links: https://pytorch-geometric.com/whl/torch-2.3.0+cu121.html


In [46]:
import networkx as nx
venuesdict = venuesdf.to_dict('records')

i = 0
nodeIds = {}
node_attr = []

for el in venuesdict:
  #print(el['Venue ID'])
  nodeIds[el['Venue ID']] = i
  node_attr.append([float(el['Venue category ID'])])
  i+=1

print(len(node_attr))


286


In [47]:
#import torch
sequences = {}
count = {}

MINVISIT = 5


for _, row in df.iterrows():

  if row['User ID'] not in sequences:
    sequences[row['User ID']] = [[row['Venue ID'],row['localHour'],row['localDayofWeek']]]


  else:
    sequences[row['User ID']].append([row['Venue ID'],row['localHour'],row['localDayofWeek']])


  if row['Venue ID'] not in count:
    count[row['Venue ID']] = 1
  else:
    count[row['Venue ID']] += 1

print(sequences)
seq = []
next = []
to_remove = []

for el in sequences:
  entry = [el]
  for i in range(len(sequences[el][len(sequences[el])-1])):
    el2 = sequences[el][len(sequences[el])-1][i]
    #print(el2,i)
    if i == 0:
      el2 = nodeIds[el2]
    #print(el2,i)
    entry.append(el2)
  for i in range(len(sequences[el])-1):
    entry.append(nodeIds[sequences[el][i][0]])
  seq.append(torch.tensor(entry))

  target = torch.zeros(numvenues)
  target[sequences[el][len(sequences[el])-1][0]] = 1
  next.append(target)
  #print(count[sequences[el][len(sequences[el])-1][0]])
  if count[sequences[el][len(sequences[el])-1][0]] < MINVISIT:

    to_remove.append(sequences[el][len(sequences[el])-1][0])
    #seq.append(torch.tensor(entry))
    #next.append(target)

seq=torch.nn.utils.rnn.pad_sequence(seq,batch_first=True)
seq_dim = len(seq[0])-4
print(seq_dim)
#seq=torch.nn.utils.rnn.pad_sequence(seq)

print(len(to_remove))

#to_remove_new = []

#for el in to_remove:

#  el = nodeIds[el]
#  to_remove_new[]

print(seq)
print(len(next))



{860: [[88, 3, 5]], 651: [[214, 20, 5], [225, 13, 6]], 201: [[84, 21, 6]], 736: [[19, 14, 3]], 154: [[62, 13, 5]], 598: [[259, 22, 4]], 980: [[278, 13, 5]], 172: [[29, 10, 0], [269, 9, 3]], 295: [[85, 3, 0]], 845: [[184, 16, 0]], 992: [[101, 9, 4]], 44: [[134, 23, 1]], 516: [[30, 12, 5]], 585: [[249, 11, 4]], 425: [[206, 19, 1]], 256: [[255, 12, 4]], 15: [[90, 15, 5]], 315: [[195, 1, 4], [146, 9, 6], [267, 5, 1]], 767: [[114, 7, 1]], 143: [[19, 6, 1]], 481: [[280, 6, 1]], 266: [[26, 0, 5]], 1039: [[48, 21, 5], [147, 18, 3]], 702: [[105, 15, 3]], 90: [[234, 9, 1], [41, 18, 4]], 93: [[121, 13, 1]], 445: [[250, 19, 1]], 573: [[145, 11, 3], [164, 17, 3]], 708: [[141, 15, 5]], 306: [[135, 10, 5]], 237: [[43, 11, 3]], 727: [[165, 15, 5]], 1054: [[218, 12, 6]], 386: [[284, 13, 1], [284, 13, 5]], 954: [[222, 14, 2]], 593: [[99, 21, 4], [154, 20, 6]], 985: [[44, 11, 3]], 131: [[14, 5, 4]], 422: [[257, 2, 3], [251, 4, 3]], 14: [[12, 19, 0]], 812: [[131, 13, 6]], 728: [[9, 17, 4]], 946: [[183, 5,

In [48]:
df = df[~df['Venue ID'].isin(to_remove)]
venuesdf = venuesdf[~venuesdf['Venue ID'].isin(to_remove)]
venuesdict = venuesdf.to_dict('records')
print(df)
print(venuesdf)

        User ID  Venue ID  Venue category ID   Latitude  Longitude  Timezone  \
117404      651       214                107  40.832115 -73.909565      -240   
94895       172        29                 11  40.760939 -73.967332      -240   
166269      315       195                 22  40.672983 -73.870710      -240   
186670     1039        48                 65  40.744923 -73.998036      -300   
225975       90       234                 90  40.755578 -73.978180      -300   
73342       573       145                 80  40.832610 -73.861045      -240   
138196      593        99                 25  40.806349 -74.192879      -240   
205723      422       257                115  40.723514 -74.224412      -300   
90948       946       183                 37  40.696337 -73.791074      -240   
36656       731       129                 48  40.752572 -73.994604      -240   
198162      472       273                113  40.740791 -73.976693      -300   
106441       53       254               

In [49]:
#fast spatial graph construction

import networkx as nx
venuesdict = venuesdf.to_dict('records')
#print(venuesdict)

#for el in venuesdict:
#  print((el,venuesdict[el]))
'''
G = nx.Graph()

distances = []

from tqdm import tqdm

for i in tqdm(range(len(venuesdict) - 1)):
    if not G.has_node(venuesdict[i]['Venue ID']):
        G.add_node(venuesdict[i]['Venue ID'], category=venuesdict[i]['Venue category ID'])

    for j in range(i + 1, len(venuesdict)):
        if not G.has_node(venuesdict[j]['Venue ID']):
            G.add_node(venuesdict[j]['Venue ID'], category=venuesdict[i]['Venue category ID'])

        if distance_between_coordinates(venuesdict[i]['Latitude'], venuesdict[i]['Longitude'],
                                        venuesdict[j]['Latitude'], venuesdict[j]['Longitude']) < 1:
            G.add_edge(venuesdict[i]['Venue ID'], venuesdict[j]['Venue ID'])
'''
from tqdm import tqdm
import torch
from torch_geometric.data import Data

edge_index = []
edges_attr = []

for i in tqdm(range(len(venuesdict) - 1)):

   #if venuesdict[i]['Venue ID'] not in node_attr:
   #  node_attr.append(venuesdict[i]['Venue ID'])

   for j in range(i + 1, len(venuesdict)):
    #  if venuesdict[j]['Venue ID'] not in node_attr:
    #     node_attr.append(venuesdict[j]['Venue ID'])


      if distance_between_coordinates(venuesdict[i]['Latitude'], venuesdict[i]['Longitude'],
                                    venuesdict[j]['Latitude'], venuesdict[j]['Longitude']) < 1:
        edge_index.append([nodeIds[venuesdict[i]['Venue ID']], nodeIds[venuesdict[j]['Venue ID']]])
        edges_attr.append([1])

spatG = Data(x=torch.tensor(node_attr), edge_index=torch.LongTensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(spatG)



100%|██████████| 57/57 [00:00<00:00, 3643.55it/s]

Data(x=[286, 1], edge_index=[2, 28], edge_attr=[28, 1])


In [50]:
import pickle

print(spatG)

#spatpG = G

pickle.dump(spatG,open('spatG.pickle','wb'))

#data = pickle.load(open('spatG.pickle','rb'))

Data(x=[286, 1], edge_index=[2, 28], edge_attr=[28, 1])


In [51]:
#nx.draw(G)
print(spatG)

Data(x=[286, 1], edge_index=[2, 28], edge_attr=[28, 1])


In [52]:

#sequantial relation graph
#build a graph that captures the sequence of actions made by the users

import networkx as nx

edge_index = []
edges_attr = []

usrloc = {}

#seqG = nx.Graph()

for _, row in df.iterrows():

  #if not seqG.has_node(row['Venue ID']):
  #  seqG.add_node(row['Venue ID'], category=row['Venue category ID'])

  if row['User ID'] not in usrloc:
    usrloc[row['User ID']] = row['Venue ID']

  else:
    #seqG.add_edge(usrloc[row['User ID']],row['Venue ID'])
    edge_index.append([nodeIds[usrloc[row['User ID']]], nodeIds[row['Venue ID']]])
    edges_attr.append(1)
    usrloc[row['User ID']] = row['Venue ID']


seqG = Data(x=torch.tensor(node_attr), edge_index=torch.tensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(seqG)






Data(x=[286, 1], edge_index=[2, 15], edge_attr=[15])


In [53]:
import pickle


print(seqG)

pickle.dump(seqG,open('tempG.pickle','wb'))

#data = pickle.load(open('tempG.pickle','rb'))

Data(x=[286, 1], edge_index=[2, 15], edge_attr=[15])


In [54]:
import math
groupeddf = df.groupby(['Venue ID','Venue category ID'],as_index=False)
avghour = groupeddf['localHour'].mean()
avgday = groupeddf['localDayofWeek'].mean()

print(avghour)

timedf = pd.concat([avghour,avgday['localDayofWeek']],axis=1)



print(timedf)

hours = {}
week = {}

#week['workdays'] = []
#week['weekend'] = []


for _, row in timedf.iterrows():

  hour = math.floor(row['localHour'])
  day = math.floor(row['localDayofWeek'])

  if hour not in hours:
    #hours[hour] = [[row['Venue ID'],row['Venue category ID']]]
    hours[hour] = [[row['Venue ID'],row['Venue category ID']]]

  else:
    hours[hour].append([row['Venue ID'],row['Venue category ID']])

  #if 1 <= day <=4:
  #  week['workdays'].append([row['Venue ID'],row['Venue category ID']])
  #else:
  #  week['weekend'].append([row['Venue ID'],row['Venue category ID']])

  if day not in week:
    #hours[hour] = [[row['Venue ID'],row['Venue category ID']]]
    week[day] = [[row['Venue ID'],row['Venue category ID']]]

  else:
    week[day].append([row['Venue ID'],row['Venue category ID']])






    Venue ID  Venue category ID  localHour
0          6                 21       21.0
1         11                 53       18.0
2         18                 53       13.0
3         28                 30        7.0
4         29                 11       10.0
5         48                 65       21.0
6         53                 35       14.0
7         57                  6       15.0
8         64                 47        7.0
9         67                 14       17.0
10        72                 21       13.0
11        77                 13        2.0
12        95                 78       21.0
13        97                108       12.0
14        98                 57       13.0
15        99                 25       21.0
16       102                 29       15.0
17       103                108       15.0
18       108                100       17.0
19       120                105       23.0
20       125                 46        5.0
21       127                 35        3.0
22       12

In [55]:
#dayG = nx.Graph()
#hourG = nx.Graph()

edge_index = []
edges_attr = []

for el in hours:
   for u in hours[el]:

    #if not hourG.has_node(u[0]):
    #  hourG.add_node(u[0], category=u[1])

    for v in hours[el]:

      if u != v:

        #if not hourG.has_node(u[0]):
        #  hourG.add_node(v[0], category=v[1])

        #hourG.add_edge(u[0],v[0])
        edge_index.append([nodeIds[u[0]], nodeIds[v[0]]])

hourG = Data(x=torch.tensor(node_attr), edge_index=torch.tensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(hourG)




Data(x=[286, 1], edge_index=[2, 132], edge_attr=[0])


In [56]:
pickle.dump(hourG,open('hourG.pickle','wb'))

#data = pickle.load(open('tempG.pickle','rb'))

In [57]:
edge_index = []
edges_attr = []


for el in week:
   for u in week[el]:

    #if not dayG.has_node(u[0]):
    #  dayG.add_node(u[0], category=u[1])

    for v in week[el]:

      if u != v:

        #if not dayG.has_node(u[0]):
        #  dayG.add_node(v[0], category=v[1])

        #dayG.add_edge(u[0],v[0])
        edge_index.append([nodeIds[u[0]], nodeIds[v[0]]])

dayG = Data(x=torch.tensor(node_attr), edge_index=torch.tensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(dayG)

Data(x=[286, 1], edge_index=[2, 476], edge_attr=[0])


In [58]:
pickle.dump(dayG,open('dayG.pickle','wb'))

In [59]:
!pip install pytorch-lightning
import pytorch_lightning as pl

In [99]:
from sklearn.model_selection import train_test_split

batchsize = 32


#train_x,test_x,train_y,test_y = train_test_split(seq,torch.stack(next),test_size=0.2,stratify=torch.stack(next))
train_x,test_x,train_y,test_y = train_test_split(seq,torch.stack(next),test_size=0.2)
#train_x,val_x,train_y,val_y = train_test_split(train_x,train_y,test_size=0.2,stratify=train_y)
train_x,val_x,train_y,val_y = train_test_split(train_x,train_y,test_size=0.2)

trainset = torch.utils.data.TensorDataset(train_x,train_y)
trainloader = torch.utils.data.DataLoader(trainset,batch_size=batchsize,shuffle=True)

valset = torch.utils.data.TensorDataset(val_x,val_y)
valloader = torch.utils.data.DataLoader(valset,batch_size=batchsize,shuffle=True)

testset = torch.utils.data.TensorDataset(test_x,test_y)
testloader = torch.utils.data.DataLoader(testset,batch_size=batchsize,shuffle=True)

In [148]:
#import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

class GCN(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 64)
        self.conv2 = GCNConv(64, out_channels)

    def forward(self, x, edge_index):
        #print(x)
        #print("sono una nuova convoluzione")
        #print(edge_index)
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

class Model(nn.Module):
    def __init__(self, input_dim, location_dim, sequence_dim):
        super(Model, self).__init__()

        # User Embedding Layer
        self.user_embedding = nn.Embedding(num_embeddings=input_dim, embedding_dim=64)

        #Last location Embedding Layer
        self.lastLoc_embedding = nn.Embedding(num_embeddings=input_dim, embedding_dim=64)

        #Time and day Input linear

        self.timeday_embedding = nn.Linear(2,64)

        # Location Embedding Layers
        self.gcn1 = GCN(spatG.num_features, 64)
        self.gcn2 = GCN(seqG.num_features, 64)
        self.gcn3 = GCN(hourG.num_features, 64)
        self.gcn4 = GCN(dayG.num_features, 64)

        # Sequence Embedding Layer
        self.self_attn = nn.MultiheadAttention(embed_dim=sequence_dim, num_heads=1)

        #Expand sequence representation

        self.expand_sequence = nn.Linear(sequence_dim,64)

        # Multihead Attention Layer
        self.multihead_attn = nn.MultiheadAttention(embed_dim=64, num_heads=8)

        # Classifier Layers
        self.linear1 = nn.Linear(2544, 128)
        self.linear2 = nn.Linear(128, 64)
        self.linear3 = nn.Linear(64, numvenues)

    def forward(self, user, lastloc, lasthour, lastday, sequence):
        #print(lastday)
        curr_batch = len(user)
        print(len(user))
        user_embed = self.user_embedding(user)

        lastloc_embed = self.lastLoc_embedding(lastloc)

        timeday_embed = self.timeday_embedding(torch.stack([lasthour.float(), lastday.float()],dim=1))

        #nodes = spatG.x[lastloc]

        #print(nodes)

        k = 1
        # Location Embeddings
        #nodes,edge_index1,mapping,_ =torch_geometric.utils.k_hop_subgraph(lastloc,k,spatG.edge_index)
        #print(len(nodes))

        #nodes = spatG.x[nodes]

        #print(nodes)

        #print(isinstance(spatG.edge_index, torch.Tensor))
        #print(spatG.edge_index)



        location_embed = self.gcn1(x=spatG.x, edge_index=spatG.edge_index)
        seq_embed = self.gcn2(x=seqG.x, edge_index=seqG.edge_index)
        hour_embed = self.gcn3(x=hourG.x,edge_index=hourG.edge_index)
        day_embed =  self.gcn4(x=dayG.x,edge_index=dayG.edge_index)

        #print(sequence)

        # Sequence Embedding
        sequence_embed, _ = self.self_attn(sequence.float(), sequence.float(), sequence.float())

        sequence_embed = self.expand_sequence(sequence_embed)

        print(user_embed.shape)
        print(location_embed.shape)
        print(seq_embed.shape)
        print(hour_embed.shape)
        print(day_embed.shape)
        print(sequence_embed.shape)
        print(lastloc_embed.shape)
        print(timeday_embed.shape)

        #user_embed = user_embed*sequence_embed

        # Concatenate all embeddings
        concat_embed = torch.cat([user_embed,lastloc_embed,location_embed,seq_embed,hour_embed,day_embed,timeday_embed,sequence_embed], dim=0)


        print(concat_embed)
        #print(len(spatG.x))

        # Multihead Attention
        attn_output, _ = self.multihead_attn(concat_embed, concat_embed, concat_embed)

        print(attn_output.shape)


        # Classifier
        out = F.relu(self.linear1(attn_output.view(curr_batch,-1)))
        out = F.relu(self.linear2(out))
        out = self.linear3(out)

        return out

# Define Lightning model using PyTorch Lightning
import pytorch_lightning as pl

class LightningModel(pl.LightningModule):
    def __init__(self, input_dim, location_dim, sequence_dim):
        super(LightningModel, self).__init__()
        self.model = Model(input_dim, location_dim, sequence_dim)

    def forward(self, user, timestamp, location, sequence):
        return self.model(user, timestamp, location, sequence)

    def training_step(self, batch, batch_idx):

        x, y = batch
        #print(x)
        y_pred = self.model(x[:,0],x[:,1],x[:,2],x[:,3],x[:,4:])
        loss = F.cross_entropy(torch.sigmoid(y_pred), y)
        #print(loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer

# Example of training the model
input_dim = 10000
location_dim = 64
sequence_dim = seq_dim
#print(seq_dim)
model = LightningModel(input_dim, location_dim, sequence_dim)

trainer = pl.Trainer(max_epochs=10)
trainer.fit(model, trainloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type  | Params
--------------------------------
0 | model | Model | 1.7 M 
--------------------------------
1.7 M     Trainable params
0         Non-trainable params
1.7 M     Total params
6.668     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

32
torch.Size([32, 64])
torch.Size([286, 64])
torch.Size([286, 64])
torch.Size([286, 64])
torch.Size([286, 64])
torch.Size([32, 64])
torch.Size([32, 64])
torch.Size([32, 64])
tensor([[ 1.9974e+00, -2.0752e+00, -3.8028e-01,  ..., -7.7028e-01,
         -4.4271e-01,  3.5068e-01],
        [ 6.0312e-04,  2.6054e-01, -3.8908e-01,  ..., -1.0715e+00,
         -1.4960e+00,  2.8442e-01],
        [ 3.6813e-01, -8.4123e-01, -5.1909e-01,  ...,  4.2894e-02,
         -9.8786e-03, -4.0241e-01],
        ...,
        [-3.3402e+01, -2.2193e+01, -2.3574e+01,  ..., -2.7914e+01,
         -2.5104e+01, -4.2208e+01],
        [-7.6839e+00, -2.9232e+00, -1.2609e+00,  ..., -6.5059e+00,
         -1.9565e+00, -6.4608e+00],
        [-7.6839e+00, -2.9232e+00, -1.2609e+00,  ..., -6.5059e+00,
         -1.9565e+00, -6.4608e+00]], grad_fn=<CatBackward0>)
torch.Size([1272, 64])
32
torch.Size([32, 64])
torch.Size([286, 64])
torch.Size([286, 64])
torch.Size([286, 64])
torch.Size([286, 64])
torch.Size([32, 64])
torch.Size([3

RuntimeError: shape '[23, -1]' is invalid for input of size 79104